In [ ]:
# import libraries
from cil.optimisation.functions import L1Norm, MixedL21Norm
from cil.optimisation.operators import GradientOperator
from cil.optimisation.algorithms import PDHG
from cil.utilities import dataexample
from cil.utilities import noise
from cil.utilities.display import show2D

import matplotlib.pyplot as plt
import numpy as np

from skimage.metrics import structural_similarity as ssim

In [ ]:
# Load a color image from the CIL gallery. 
data = dataexample.RAINBOW.get(size=(512,512))
data.reorder(['horizontal_y', 'horizontal_x','channel'])
ig = data.geometry

# Add gaussian noise
noisy_data = noise.saltnpepper(data, amount=0.1, seed = 10)

In [ ]:
# Setup and run the PDHG algorithm
alpha = 1.
F = alpha*MixedL21Norm()
G = L1Norm(b=noisy_data)
K = GradientOperator(ig)

# Compute operator Norm
normK = K.norm()

# Primal & dual stepsizes
sigma = 1./normK
tau = 1./normK


pdhg = PDHG(f=F, g=G, operator=K, tau=tau, sigma=sigma,
            max_iteration = 1000, update_objective_interval = 200)
pdhg.run(verbose=2)

In [ ]:
val_ssim = ssim(data.as_array(), pdhg.solution.as_array(), data_range = 1., multichannel=True)

In [ ]:
fig, ax = plt.subplots(nrows=2,ncols=2, figsize=(10,10))
ax[0,0].imshow(data.as_array())
ax[0,0].set_title("Ground Truth")

ax[0,1].imshow(noisy_data.as_array())
ax[0,1].set_title("Corrupted Data (Gaussian)")
ax[1,0].imshow(pdhg.solution.as_array())
ax[1,0].set_title("Total variation restoration (SSIM = {:.2f}): alpha = {}".format(val_ssim, alpha))
ax[1,1].imshow((pdhg.solution-data).abs().as_array())
ax[1,1].set_title("Absolute Difference")
plt.tight_layout()
plt.show()